In [4]:
import HyperParameters as hp

hp.RandomForestHyperParameters()

,max_depth,n_estimators
0,3,200
1,3,200
2,3,200
3,3,200
4,3,200
...,...,...
395,10,1000
396,10,1000
397,10,1000
398,10,1000


In [5]:
import pandas as pd
import numpy as np
import EvaluateModels as em
import boto3

## defining bucket
s3 = boto3.resource('s3')

bucket_name = 'gabriel-de-medeiros'

bucket = s3.Bucket(bucket_name)

## specifying file from the bucket
file_key = 'CleanTrainData.csv'
file_key2 = 'CleanTestData.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')


## Reading csv file, index_col = 0 makes the first column of the data to become the index of our pandas data frame
train_data = pd.read_csv(file_content_stream, index_col = 0)
test_data = pd.read_csv(file_content_stream2, index_col = 0)

train_data = train_data.reset_index(drop = True)
test_data = test_data.reset_index(drop = True)

train_data = train_data.select_dtypes(exclude=['object'])
test_data = test_data.select_dtypes(exclude=['object'])

X_t = train_data.dropna()
X_tst = test_data.dropna()


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:

train_data = X_t.iloc[0:10000,:]
test_data = X_tst.iloc[0:10000,:]


X_train, Y_train = train_data.drop(columns = 'is_fraud'), train_data['is_fraud']


X_train.columns

Index(['uses_per_category', 'std_dist', 'std_dist_by_category',
       'std_amt_by_category', 'std_amt', 'avg_amt', 'avg_by_category',
       'total_uses', 'hour_of_the_day', 'uses_per_hour', 'month',
       'uses_per_month', 'uses_per_day', 'avg_distance',
       'avg_distance_by_category', 'amt', 'zip', 'lat', 'long', 'city_pop',
       'unix_time', 'merch_lat', 'merch_long', 'distance', 'age',
       'diff_by_card_trans', 'diff_first_last', 'purchase_>_avg',
       'purchase_>_avg_by_category', 'purchase_>_distance',
       'purchase_>_distance_by_category', 'amt_>_1_std', 'amt_>_2_std',
       'amt_>_1_std_by_cagegory', 'amt_>_2_std_by_cagegory', 'dist_>_1_std',
       'dist_>_2_std', 'dist_>_1_std_by_cagegory', 'dist_>_2_std_by_cagegory'],
      dtype='object')

In [ ]:
X = X_train
Y = Y_train

import Get_Variable_Importance as gvi

print('DecisionTrees')
decision_trees_variable_importance = gvi.Decision_Tree_Importance(X,Y)

print('RandomForest')
random_forest_variable_importance = gvi.Random_Forest_Importance(X,Y)

print('ImportantVariables')
df_importance_columns = gvi.Getting_Best_Model(decision_trees_variable_importance,random_forest_variable_importance,X,Y)

DecisionTrees


100%|██████████| 80/80 [01:02<00:00,  1.28it/s]


RandomForest


 34%|███▍      | 274/800 [02:28<05:32,  1.58it/s]

In [ ]:
X_test = test_data[df_importance_columns.drop(columns = 'is_fraud').columns]
Y_test = test_data['is_fraud']

X_train = train_data[df_importance_columns.drop(columns = 'is_fraud').columns]
Y_train = train_data['is_fraud']

In [7]:
train_data = X_t.iloc[0:10000,:]
test_data = X_tst.iloc[0:10000,:]


X_test = test_data.drop(columns = 'is_fraud')
X_train = train_data.drop(columns = 'is_fraud')
Y_test = test_data['is_fraud']
Y_train = train_data['is_fraud']

In [8]:
## Decision Trees
DTC_results = em.DecisionTreesResults(X_test, X_train, Y_test, Y_train)
DTC_best_model = em.DecisionTreesBestModel(DTC_results)

DecisionTrees


100%|██████████| 8/8 [00:01<00:00,  4.28it/s]


In [9]:
DTC_results

,max_depth,cut_off,accuracy,recall,Performance
0,3,0.10,0.9272,0.789474,0.852812
1,3,0.15,0.9429,0.657895,0.775026
2,3,0.20,0.9429,0.657895,0.775026
3,3,0.25,0.9429,0.657895,0.775026
4,3,0.30,0.9429,0.657895,0.775026
...,...,...,...,...,...
75,10,0.35,0.9928,0.552632,0.710032
76,10,0.40,0.9928,0.552632,0.710032
77,10,0.45,0.9928,0.552632,0.710032
78,10,0.50,0.9928,0.552632,0.710032


In [ ]:
DTC_best_model

In [ ]:
LR_results = em.LogisticRegressionResults(X_test, X_train, Y_test, Y_train)

In [ ]:
LR_results

In [ ]:
NN_results = em.NeuralNetworksResults(X_test, X_train, Y_test, Y_train)

In [ ]:
RF_results = em.RandomForestResults(X_test, X_train, Y_test, Y_train)
RF_best_model = em.DecisionTreesBestModel(RF_results)

In [ ]:
## SVC
SVC_results = em.SupportVectorMachineResults(X_test, X_train, Y_test, Y_train)
SVC_best_model = em.SvcBestModel(SVC_results)

In [ ]:
## AdaBoost Decision Trees
ADA_DTC = em.AdaBoostDecisionTreesResults(X_test, X_train, Y_test, Y_train, DTC_best_model)



In [ ]:
## AdaBoost SVC
ADA_SVC= em.AdaBoostSvmResults(X_test, X_train, Y_test, Y_train, SVC_best_model)



In [ ]:
# GradientBoosting
GBC_results = em.GradientBoostingResults(X_test, X_train, Y_test, Y_train)

In [ ]:




# DTC_results.to_csv('CreditCardsFraudDetection/Algorithm/DTC_results.csv', index = False)
# DTC_best_model.to_csv('CreditCardsFraudDetection/Algorithm/DTC_best_model.csv', index = False)
# RF_results.to_csv('CreditCardsFraudDetection/Algorithm/RF_results.csv', index = False)
# RF_best_model.to_csv('CreditCardsFraudDetection/Algorithm/RF_best_model.csv', index = False)
# NN_results.to_csv('CreditCardsFraudDetection/Algorithm/NN_results.csv', index = False)
# SVC_results.to_csv('CreditCardsFraudDetection/Algorithm/SVC_results.csv', index = False)
# SVC_best_model.to_csv('CreditCardsFraudDetection/Algorithm/SVC_best_model.csv', index = False)
# LR_results.to_csv('CreditCardsFraudDetection/Algorithm/LR_results.csv', index = False)
# ADA_DTC.to_csv('CreditCardsFraudDetection/Algorithm/ADA_DTC.csv', index = False)
# ADA_SVC.to_csv('CreditCardsFraudDetection/Algorithm/ADA_SVC.csv', index = False)
# GBC_results.to_csv('CreditCardsFraudDetection/Algorithm/GBC_results.csv', index = False)